In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
xiaoweixumedicalai_chd68_segmentation_dataset_miccai19_path = kagglehub.dataset_download('xiaoweixumedicalai/chd68-segmentation-dataset-miccai19')
xiaoweixumedicalai_imagechd_path = kagglehub.dataset_download('xiaoweixumedicalai/imagechd')
azizesseghaier_parse_challenge_2022_training_set_path = kagglehub.dataset_download('azizesseghaier/parse-challenge-2022-training-set')
azizesseghaier_image_chd_pa_only_labels_path = kagglehub.dataset_download('azizesseghaier/image-chd-pa-only-labels')

print('Data source import complete.')


# Unzip the image chd dataset and keep only the pulmonary arteries labels

In [ ]:
import os
import nibabel as nib
import numpy as np
from pathlib import Path

In [ ]:
os.makedirs("imagechd", exist_ok=True)

In [ ]:
!mv /kaggle/input/imagechd/ImageCHD_dataset.change2zip /kaggle/working/imagechd/ImageCHD_dataset.zip
!mv /kaggle/input/imagechd/ImageCHD_dataset.z01 /kaggle/working/imagechd/ImageCHD_dataset.z01
!mv /kaggle/input/imagechd/ImageCHD_dataset.z02 /kaggle/working/imagechd/ImageCHD_dataset.z02
!mv /kaggle/input/imagechd/ImageCHD_dataset.z03 /kaggle/working/imagechd/ImageCHD_dataset.z03
!mv /kaggle/input/imagechd/ImageCHD_dataset.z04 /kaggle/working/imagechd/ImageCHD_dataset.z04
!mv /kaggle/input/imagechd/ImageCHD_dataset.z05 /kaggle/working/imagechd/ImageCHD_dataset.z05
!mv /kaggle/input/imagechd/ImageCHD_dataset.z06 /kaggle/working/imagechd/ImageCHD_dataset.z06
!mv /kaggle/input/imagechd/ImageCHD_dataset.z07 /kaggle/working/imagechd/ImageCHD_dataset.z07
!mv /kaggle/input/imagechd/ImageCHD_dataset.z08 /kaggle/working/imagechd/ImageCHD_dataset.z08
!mv /kaggle/input/imagechd/ImageCHD_dataset.z09 /kaggle/working/imagechd/ImageCHD_dataset.z09
!mv /kaggle/input/imagechd/ImageCHD_dataset.z10 /kaggle/working/imagechd/ImageCHD_dataset.z10
!mv /kaggle/input/imagechd/ImageCHD_dataset.z11 /kaggle/working/imagechd/ImageCHD_dataset.z11
!mv /kaggle/input/imagechd/ImageCHD_dataset.z12 /kaggle/working/imagechd/ImageCHD_dataset.z12

In [ ]:
#!cat /kaggle/working/chd68-segmentation-dataset-miccai19/CHD68_segmentation_dataset_miccai19.z* /kaggle/working/chd68-segmentation-dataset-miccai19/CHD68_segmentation_dataset_miccai19.zip > /kaggle/working/full_dataset.zip

In [ ]:
!apt-get install -y p7zip-full
!7z x /kaggle/working/imagechd/ImageCHD_dataset.zip -o/kaggle/working/unzipped

In [ ]:
!rm -rf /kaggle/working/imagechd

## Extracting all possible labels from the image chd dataset

In [ ]:
#extracting all possible labels and their data types

# Directory containing your label files
source_dir = Path("/kaggle/working/unzipped/ImageCHD_dataset")

data_type_set = set()
labels_set = set()

# Loop over all files ending with _label.nii or _label.nii.gz
for label_path in sorted(source_dir.glob("*_label.nii*")):
    # Skip files starting with ._
    if label_path.name.startswith("._"):
        continue

    print("Processing:", label_path.name)
    img = nib.load(label_path)
    data = img.get_fdata()

    # Collect info
    labels = np.unique(data)
    data_type_set.add(data.dtype)
    labels_set = labels_set.union(set(labels))

print("Data types found:", data_type_set)
print("All labels found:", labels_set)

## Keeping the pulmonary arteries labels

In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Directory containing both images and labels
data_dir = Path("/kaggle/working/unzipped/ImageCHD_dataset")

# Output directory
output_dir = Path("/kaggle/working/image_chd_pa_only_labels/")
output_dir.mkdir(exist_ok=True)

PATHS = sorted(data_dir.glob("*_label.nii*"))
# Loop over all label files
for label_path in tqdm(PATHS, desc="processing labels"):
    # Skip hidden files
    if label_path.name.startswith("._"):
        continue

    print("Processing:", label_path.name)

    # Load label file
    label_img = nib.load(label_path)
    label_data = label_img.get_fdata()

    # Extract pulmonary artery (label 7) and convert to int8
    pulmonary_only = np.where(label_data == 7., 1.0, 0.0)

    # Skip saving if pulmonary artery not present
    if np.max(pulmonary_only) == 0:
        print("No pulmonary artery label found, skipping.")
        continue

    # Create new NIfTI image for pulmonary label
    new_label_img = nib.Nifti1Image(pulmonary_only, label_img.affine, label_img.header)
    label_out_path = output_dir / label_path.name
    nib.save(new_label_img, label_out_path)

print("Done!!")


In [ ]:
#! zip -r labels.zip /kaggle/working/image_chd_pa_only_labels

# Checking all possible labels in the parse dataset

In [ ]:
import nibabel as nib
import numpy as np

img = nib.load("/kaggle/input/parse-challenge-2022-training-set/train/train/PA000036/label/PA000036.nii")
data = img.get_fdata()
labels = np.unique(data)
labels_set = set(labels)
labels_set

In [ ]:
all_poosible_labels_CHD68 = {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 11.0, 12.0, 13.0, 14.0, 15.0}

In [ ]:
all_possible_labels_imageCHD = {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 10.0, 12.0, 13.0, 14.0, 15.0}

# Checking for possible duplicates in both datasets

In [ ]:
"""
Notebook-friendly duplicate checker for quick analysis in Kaggle/Colab

Usage in notebook:
    %run scripts/check_duplicates_notebook.py

    Or import:
    from scripts.check_duplicates_notebook import quick_duplicate_check

    results = quick_duplicate_check(
        parse22_path="/kaggle/input/parse-challenge-2022-training-set",
        imagechd_path="/kaggle/working/unzipped/ImageCHD_dataset"
    )
"""

import hashlib
import numpy as np
import SimpleITK as sitk
from pathlib import Path
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd


def find_parse22_images(base_path):
    """Find PARSE22 images."""
    base_path = Path(base_path)
    images = []

    # Try different patterns
    for pattern in ["train/train/*/image/*.nii", "**/PA*/image/*.nii"]:
        images.extend(base_path.glob(pattern))

    images = list(set(images))
    print(f"✓ Found {len(images)} PARSE22 images")
    return images


def find_imagechd_images(base_path):
    """Find ImageCHD images."""
    base_path = Path(base_path)
    images = []

    # Try different patterns
    for pattern in ["ct_*_image.nii.gz", "**/ct_*_image.nii.gz"]:
        images.extend(base_path.glob(pattern))

    images = list(set(images))
    print(f"✓ Found {len(images)} ImageCHD images")
    return images


def compute_file_hash(file_path):
    """Compute MD5 hash of file."""
    md5_hash = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            md5_hash.update(chunk)
    return md5_hash.hexdigest()


def compute_image_hash(image_array):
    """Compute hash of image content."""
    # Normalize and quantize
    normalized = (image_array - image_array.min()) / (image_array.max() - image_array.min() + 1e-10)
    quantized = (normalized * 1000).astype(np.int16)
    return hashlib.md5(quantized.tobytes()).hexdigest()


def load_image_info(file_path):
    """Load image and extract basic info."""
    try:
        img = sitk.ReadImage(str(file_path))
        array = sitk.GetArrayFromImage(img)

        return {
            'path': str(file_path),
            'name': file_path.name,
            'shape': array.shape,
            'spacing': img.GetSpacing(),
            'size': img.GetSize(),
            'min': float(array.min()),
            'max': float(array.max()),
            'mean': float(array.mean()),
            'std': float(array.std()),
            'array': array  # Keep for hash computation
        }
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None


def check_file_duplicates(parse22_images, imagechd_images):
    """Check for exact file duplicates."""
    print("\n" + "="*60)
    print("Checking file hashes (exact file duplicates)...")
    print("="*60)

    hash_map = defaultdict(list)

    all_images = parse22_images + imagechd_images

    for img_path in tqdm(all_images, desc="Computing file hashes"):
        file_hash = compute_file_hash(img_path)
        hash_map[file_hash].append(str(img_path))

    # Find duplicates
    duplicates = {h: files for h, files in hash_map.items() if len(files) > 1}

    print(f"\nResult: Found {len(duplicates)} groups of file duplicates")

    if duplicates:
        print("\nDuplicate groups:")
        for i, (hash_val, files) in enumerate(duplicates.items(), 1):
            print(f"\n  Group {i} ({len(files)} files):")
            for f in files:
                print(f"    - {Path(f).name}")

    return duplicates


def check_content_duplicates(parse22_images, imagechd_images):
    """Check for content duplicates (same pixel values)."""
    print("\n" + "="*60)
    print("Checking content hashes (identical pixel values)...")
    print("="*60)

    hash_map = defaultdict(list)
    image_info = {}

    all_images = parse22_images + imagechd_images

    for img_path in tqdm(all_images, desc="Loading images and computing hashes"):
        info = load_image_info(img_path)
        if info is None:
            continue

        # Compute content hash
        content_hash = compute_image_hash(info['array'])
        hash_map[content_hash].append(info)
        image_info[str(img_path)] = info

        # Remove array to save memory
        del info['array']

    # Find duplicates
    duplicates = {h: infos for h, infos in hash_map.items() if len(infos) > 1}

    print(f"\nResult: Found {len(duplicates)} groups of content duplicates")

    if duplicates:
        print("\nDuplicate groups:")
        for i, (hash_val, infos) in enumerate(duplicates.items(), 1):
            print(f"\n  Group {i} ({len(infos)} images):")
            for info in infos:
                print(f"    - {info['name']}: shape={info['shape']}, mean={info['mean']:.1f}")

    return duplicates, image_info


def check_metadata_similarity(image_info):
    """Check for potential duplicates based on metadata."""
    print("\n" + "="*60)
    print("Checking metadata similarity...")
    print("="*60)

    # Group by shape
    shape_groups = defaultdict(list)
    for path, info in image_info.items():
        if 'array' in info:
            del info['array']  # Clean up
        shape_groups[str(info['shape'])].append(info)

    # Find cross-dataset matches
    potential_duplicates = []

    for shape, images in shape_groups.items():
        if len(images) < 2:
            continue

        # Separate by dataset
        parse22 = [img for img in images if 'PA' in img['name']]
        imagechd = [img for img in images if 'ct_' in img['name']]

        if not parse22 or not imagechd:
            continue

        # Compare metadata
        for p22 in parse22:
            for ichd in imagechd:
                mean_diff = abs(p22['mean'] - ichd['mean'])
                std_diff = abs(p22['std'] - ichd['std'])

                # Flag if very similar
                if mean_diff < 50 and std_diff < 50:
                    potential_duplicates.append({
                        'parse22': p22['name'],
                        'imagechd': ichd['name'],
                        'shape': p22['shape'],
                        'mean_diff': mean_diff,
                        'std_diff': std_diff,
                        'p22_mean': p22['mean'],
                        'ichd_mean': ichd['mean']
                    })

    print(f"\nResult: Found {len(potential_duplicates)} potential duplicate pairs")

    if potential_duplicates:
        print("\nTop 5 suspicious pairs:")
        for i, dup in enumerate(sorted(potential_duplicates, key=lambda x: x['mean_diff'])[:5], 1):
            print(f"\n  Pair {i}:")
            print(f"    PARSE22: {dup['parse22']}")
            print(f"    ImageCHD: {dup['imagechd']}")
            print(f"    Mean difference: {dup['mean_diff']:.2f}")

    return potential_duplicates


def create_summary_dataframe(image_info):
    """Create summary DataFrame of all images."""
    data = []

    for path, info in image_info.items():
        if 'array' in info:
            continue

        dataset = 'PARSE22' if 'PA' in info['name'] else 'ImageCHD'
        data.append({
            'dataset': dataset,
            'name': info['name'],
            'shape': str(info['shape']),
            'spacing': str(info['spacing']),
            'min': info['min'],
            'max': info['max'],
            'mean': info['mean'],
            'std': info['std']
        })

    df = pd.DataFrame(data)
    return df


def quick_duplicate_check(parse22_path, imagechd_path, skip_metadata=False):
    """
    Quick duplicate check - returns results dictionary.

    Args:
        parse22_path: Path to PARSE22 dataset
        imagechd_path: Path to ImageCHD dataset
        skip_metadata: Skip metadata similarity check (faster)

    Returns:
        dict: Results including duplicates and image info
    """
    print("="*60)
    print("QUICK DUPLICATE CHECK")
    print("="*60)
    print(f"PARSE22: {parse22_path}")
    print(f"ImageCHD: {imagechd_path}")
    print()

    # Find images
    parse22_images = find_parse22_images(parse22_path)
    imagechd_images = find_imagechd_images(imagechd_path)

    if not parse22_images or not imagechd_images:
        print("ERROR: Could not find images in one or both datasets!")
        return None

    # Check duplicates
    file_duplicates = check_file_duplicates(parse22_images, imagechd_images)
    content_duplicates, image_info = check_content_duplicates(parse22_images, imagechd_images)

    potential_duplicates = []
    if not skip_metadata:
        potential_duplicates = check_metadata_similarity(image_info)

    # Summary
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Total images: {len(parse22_images) + len(imagechd_images)}")
    print(f"  PARSE22: {len(parse22_images)}")
    print(f"  ImageCHD: {len(imagechd_images)}")
    print()
    print(f"Exact file duplicates: {len(file_duplicates)} groups")
    print(f"Exact content duplicates: {len(content_duplicates)} groups")
    print(f"Potential duplicates: {len(potential_duplicates)} pairs")
    print()

    # Create summary DataFrame
    df_summary = create_summary_dataframe(image_info)

    # Recommendation
    total_exact = len(file_duplicates) + len(content_duplicates)
    if total_exact == 0:
        print("✓ No exact duplicates found!")
        print("  All images appear to be unique.")
        expected_total = len(parse22_images) + len(imagechd_images)
    else:
        print("⚠ Found exact duplicates!")
        print("  Recommend removing duplicates before training.")
        # Estimate unique images
        dup_count = sum(len(files) - 1 for files in file_duplicates.values())
        dup_count += sum(len(infos) - 1 for infos in content_duplicates.values())
        expected_total = len(parse22_images) + len(imagechd_images) - dup_count

    print(f"\nExpected dataset size after cleanup: {expected_total} images")
    print("="*60)

    return {
        'parse22_images': parse22_images,
        'imagechd_images': imagechd_images,
        'file_duplicates': file_duplicates,
        'content_duplicates': content_duplicates,
        'potential_duplicates': potential_duplicates,
        'image_info': image_info,
        'summary_df': df_summary,
        'total_images': len(parse22_images) + len(imagechd_images),
        'expected_unique': expected_total
    }


# Example usage for Kaggle/Colab
if __name__ == "__main__":
    # Example paths (adjust for your environment)
    PARSE22_PATH = "/kaggle/input/parse-challenge-2022-training-set"
    IMAGECHD_PATH = "/kaggle/working/unzipped/ImageCHD_dataset"

    # Run check
    results = quick_duplicate_check(PARSE22_PATH, IMAGECHD_PATH)

    if results:
        # Display summary DataFrame
        print("\nDataset Summary:")
        print(results['summary_df'].head(10))

        # Save results
        results['summary_df'].to_csv('dataset_summary.csv', index=False)
        print("\nSaved dataset summary to: dataset_summary.csv")

# Grouping images by dimension

In [ ]:
"""
Simple dimension checker optimized for Kaggle notebooks.
Fast, lightweight, and produces easy-to-read output.

Copy-paste this entire cell into your Kaggle notebook and run!
"""

import SimpleITK as sitk
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')


def quick_dimension_analysis(parse22_path, imagechd_path):
    """
    Quick dimension analysis - groups images by (D, H, W, C).

    Args:
        parse22_path: Path to PARSE22 dataset
        imagechd_path: Path to ImageCHD dataset

    Returns:
        dict: Results with dimension groups
    """
    print("="*70)
    print("DATASET DIMENSION ANALYZER")
    print("="*70)

    # Find images
    print("\n🔍 Finding images...")
    parse22_images = list(Path(parse22_path).glob("**/PA*/image/*.nii*"))
    imagechd_images = list(Path(imagechd_path).glob("**/ct_*_image.nii*"))

    print(f"  PARSE22: {len(parse22_images)} images")
    print(f"  ImageCHD: {len(imagechd_images)} images")
    print(f"  Total: {len(parse22_images) + len(imagechd_images)} images\n")

    # Analyze dimensions
    dimension_groups = defaultdict(lambda: {'parse22': [], 'imagechd': [], 'spacing': []})
    all_data = []

    print("📊 Analyzing PARSE22...")
    for img_path in tqdm(parse22_images):
        try:
            img = sitk.ReadImage(str(img_path))
            array = sitk.GetArrayFromImage(img)
            spacing = img.GetSpacing()

            # Get dimensions: array shape is (D, H, W)
            d, h, w = array.shape
            c = 1  # channels

            dim_key = (d, h, w, c)
            dimension_groups[dim_key]['parse22'].append(img_path.name)
            dimension_groups[dim_key]['spacing'].append(spacing)

            all_data.append({
                'file': img_path.name,
                'dataset': 'PARSE22',
                'D': d, 'H': h, 'W': w, 'C': c,
                'spacing_x': spacing[0],
                'spacing_y': spacing[1],
                'spacing_z': spacing[2],
                'voxels': d*h*w,
                'memory_mb': array.nbytes / (1024**2)
            })
        except Exception as e:
            print(f"  ⚠️  Error with {img_path.name}: {e}")

    print("\n📊 Analyzing ImageCHD...")
    for img_path in tqdm(imagechd_images):
        try:
            img = sitk.ReadImage(str(img_path))
            array = sitk.GetArrayFromImage(img)
            spacing = img.GetSpacing()

            d, h, w = array.shape
            c = 1

            dim_key = (d, h, w, c)
            dimension_groups[dim_key]['imagechd'].append(img_path.name)
            dimension_groups[dim_key]['spacing'].append(spacing)

            all_data.append({
                'file': img_path.name,
                'dataset': 'ImageCHD',
                'D': d, 'H': h, 'W': w, 'C': c,
                'spacing_x': spacing[0],
                'spacing_y': spacing[1],
                'spacing_z': spacing[2],
                'voxels': d*h*w,
                'memory_mb': array.nbytes / (1024**2)
            })
        except Exception as e:
            print(f"  ⚠️  Error with {img_path.name}: {e}")

    # Display results
    print("\n" + "="*70)
    print("DIMENSION GROUPS (D × H × W × Channels)")
    print("="*70)

    # Sort by total count
    sorted_groups = sorted(
        dimension_groups.items(),
        key=lambda x: len(x[1]['parse22']) + len(x[1]['imagechd']),
        reverse=True
    )

    for i, (dims, data) in enumerate(sorted_groups, 1):
        d, h, w, c = dims
        p22_count = len(data['parse22'])
        ichd_count = len(data['imagechd'])
        total = p22_count + ichd_count

        # Get spacing statistics
        if data['spacing']:
            # Average spacing
            avg_spacing = tuple(
                sum(s[i] for s in data['spacing']) / len(data['spacing'])
                for i in range(3)
            )

            # Most frequent spacing (rounded to 2 decimals)
            from collections import Counter
            spacing_rounded = [tuple(round(s[i], 2) for i in range(3)) for s in data['spacing']]
            most_common_spacing = Counter(spacing_rounded).most_common(1)[0][0]

            avg_spacing_str = f"{avg_spacing[2]:.2f} × {avg_spacing[1]:.2f} × {avg_spacing[0]:.2f} mm"
            mode_spacing_str = f"{most_common_spacing[2]:.2f} × {most_common_spacing[1]:.2f} × {most_common_spacing[0]:.2f} mm"
        else:
            avg_spacing_str = "N/A"
            mode_spacing_str = "N/A"

        print(f"\n{'='*70}")
        print(f"Group {i}: {d} × {h} × {w} (channels: {c})")
        print(f"{'='*70}")
        print(f"Total images: {total}")
        print(f"  • PARSE22: {p22_count} images")
        print(f"  • ImageCHD: {ichd_count} images")
        print(f"Average spacing: {avg_spacing_str}")
        print(f"Most frequent spacing: {mode_spacing_str}")
        print(f"Voxel count: {d * h * w:,}")

        # Show sample files
        print(f"\nSample files:")
        if data['parse22']:
            print(f"  PARSE22:")
            for name in data['parse22'][:3]:
                print(f"    - {name}")
            if p22_count > 3:
                print(f"    ... and {p22_count - 3} more")

        if data['imagechd']:
            print(f"  ImageCHD:")
            for name in data['imagechd'][:3]:
                print(f"    - {name}")
            if ichd_count > 3:
                print(f"    ... and {ichd_count - 3} more")

    # Create DataFrame
    df = pd.DataFrame(all_data)

    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    print(f"Total images: {len(df)}")
    print(f"Unique dimension combinations: {len(dimension_groups)}")

    # Dimension statistics by dataset
    print(f"\n{'='*70}")
    print("DIMENSION STATISTICS BY DATASET")
    print(f"{'='*70}")

    for dataset_name in ['PARSE22', 'ImageCHD']:
        dataset_df = df[df['dataset'] == dataset_name]
        if len(dataset_df) == 0:
            continue

        print(f"\n{dataset_name}:")
        print(f"  Count: {len(dataset_df)} images")

        # Depth (D) statistics
        d_mean = dataset_df['D'].mean()
        d_mode = dataset_df['D'].mode().values[0] if len(dataset_df['D'].mode()) > 0 else "N/A"
        d_min = dataset_df['D'].min()
        d_max = dataset_df['D'].max()
        print(f"  Depth (D):")
        print(f"    • Average: {d_mean:.1f} slices")
        print(f"    • Most frequent: {d_mode} slices")
        print(f"    • Range: [{d_min}, {d_max}]")

        # Height (H) statistics
        h_mean = dataset_df['H'].mean()
        h_mode = dataset_df['H'].mode().values[0] if len(dataset_df['H'].mode()) > 0 else "N/A"
        h_min = dataset_df['H'].min()
        h_max = dataset_df['H'].max()
        print(f"  Height (H):")
        print(f"    • Average: {h_mean:.1f} pixels")
        print(f"    • Most frequent: {h_mode} pixels")
        print(f"    • Range: [{h_min}, {h_max}]")

        # Width (W) statistics
        w_mean = dataset_df['W'].mean()
        w_mode = dataset_df['W'].mode().values[0] if len(dataset_df['W'].mode()) > 0 else "N/A"
        w_min = dataset_df['W'].min()
        w_max = dataset_df['W'].max()
        print(f"  Width (W):")
        print(f"    • Average: {w_mean:.1f} pixels")
        print(f"    • Most frequent: {w_mode} pixels")
        print(f"    • Range: [{w_min}, {w_max}]")

        # Spacing statistics
        print(f"  Spacing:")
        print(f"    • X (width) - Avg: {dataset_df['spacing_x'].mean():.3f} mm, Mode: {dataset_df['spacing_x'].mode().values[0]:.3f} mm")
        print(f"    • Y (height) - Avg: {dataset_df['spacing_y'].mean():.3f} mm, Mode: {dataset_df['spacing_y'].mode().values[0]:.3f} mm")
        print(f"    • Z (depth) - Avg: {dataset_df['spacing_z'].mean():.3f} mm, Mode: {dataset_df['spacing_z'].mode().values[0]:.3f} mm")

    print(f"\n{'='*70}")
    print("OVERALL STATISTICS")
    print(f"{'='*70}")
    print(f"\nDimension distribution:")
    print(df.groupby(['D', 'H', 'W']).size().sort_values(ascending=False))

    # Memory analysis
    print(f"\nMemory requirements:")
    print(f"  Average per image: {df['memory_mb'].mean():.1f} MB")
    print(f"  Total dataset: {df['memory_mb'].sum():.1f} MB")
    print(f"  Min: {df['memory_mb'].min():.1f} MB")
    print(f"  Max: {df['memory_mb'].max():.1f} MB")

    return {
        'dimension_groups': dict(dimension_groups),
        'dataframe': df,
        'summary': {
            'total_images': len(df),
            'parse22_count': len(df[df['dataset'] == 'PARSE22']),
            'imagechd_count': len(df[df['dataset'] == 'ImageCHD']),
            'unique_dimensions': len(dimension_groups)
        }
    }


def print_dimension_lookup(dimension_groups):
    """
    Print a clean lookup table for dimensions.
    """
    print("\n" + "="*70)
    print("QUICK REFERENCE TABLE")
    print("="*70)

    data = []
    for dims, info in dimension_groups.items():
        d, h, w, c = dims
        p22 = len(info['parse22'])
        ichd = len(info['imagechd'])
        total = p22 + ichd

        data.append({
            'Dimensions (D×H×W)': f"{d}×{h}×{w}",
            'PARSE22': p22,
            'ImageCHD': ichd,
            'Total': total,
            'Percentage': f"{(total/sum(len(i['parse22'])+len(i['imagechd']) for i in dimension_groups.values()))*100:.1f}%"
        })

    df = pd.DataFrame(data).sort_values('Total', ascending=False)
    print(df.to_string(index=False))
    print()

    return df


def get_files_by_dimension(dimension_groups, target_dims):
    """
    Get all files with specific dimensions.

    Args:
        dimension_groups: Results from quick_dimension_analysis
        target_dims: Tuple of (D, H, W, C) or (D, H, W)

    Returns:
        dict: Files grouped by dataset
    """
    # Handle 3-tuple input
    if len(target_dims) == 3:
        target_dims = (*target_dims, 1)

    if target_dims in dimension_groups:
        return dimension_groups[target_dims]
    else:
        print(f"Dimension {target_dims} not found!")
        print("Available dimensions:")
        for dims in dimension_groups.keys():
            print(f"  {dims}")
        return None


# ============================================================================
# EXAMPLE USAGE IN KAGGLE NOTEBOOK
# ============================================================================

if __name__ == "__main__":
    # Set your paths
    PARSE22_PATH = "/kaggle/input/parse-challenge-2022-training-set"
    IMAGECHD_PATH = "/kaggle/working/unzipped/ImageCHD_dataset"

    # Run analysis
    results = quick_dimension_analysis(PARSE22_PATH, IMAGECHD_PATH)

    # Print lookup table
    #lookup_df = print_dimension_lookup(results['dimension_groups'])

    # Save results
    #results['dataframe'].to_csv('dataset_dimensions.csv', index=False)
    #print("💾 Saved: dataset_dimensions.csv")